In [13]:
import pandas as pd
base_path="/home/jacob/data/movies/dailyboxoffice"
ds_nodash="20240101"
svbase_path =  "/home/jacob/data/movies/merge/dailyboxoffice"
save_path = f"{svbase_path}/dt={ds_nodash}/merged.parquet"
drop_col = ['rnum', 'rank', 'rankInten', 'salesShare']
full_col = ['rnum', 'rank', 'rankInten', 'salesAmt', 'audiCnt',
            'audiAcc', 'scrnCnt', 'showCnt', 'salesShare', 'salesInten',
            'salesChange', 'audiInten', 'audiChange']
    
df=pd.read_parquet("/home/jacob/data/movies/dailyboxoffice/dt=20240101")
df.drop(columns=['rank', 'rnum', 'rankInten', 'salesShare'])
fil_movieCd=[]

for _, row in df.iterrows():
    if pd.isna(row['multiMovieYn']) or pd.isna(row['repNationCd']):
        fil_movieCd.append(row['movieCd'])
        
def merge_values(series):
    return ', '.join(series.dropna().astype(str).unique())

merged_list=[]

for i in set(fil_movieCd):
    fil_dup=df[df['movieCd'] == i][['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd','audiCnt','rnum']]
    if len(fil_dup) == 1 and fil_dup[['multiMovieYn', 'repNationCd']].isna().all(axis=1).iloc[0]:
        fil_dup = fil_dup.fillna("Unclassified")
        merged_list.append(fil_dup)
    else:
        fil_dup=fil_dup.dropna(subset=['multiMovieYn', 'repNationCd'], how='all')
        merged_df = fil_dup.groupby(['movieCd', 'movieNm'], as_index=False).agg({
            'multiMovieYn': merge_values,
            'repNationCd': merge_values,
            'audiCnt': 'max'
        })
        merged_list.append(merged_df)
f_merged_df = pd.concat(merged_list, ignore_index=True)
f_merged_df['rank'] =f_merged_df['audiCnt'].rank(ascending=False,method='dense')
unique_df_sorted = f_merged_df.sort_values(by='rank')
unique_df_sorted[['multiMovieYn', 'repNationCd']] = unique_df_sorted[['multiMovieYn', 'repNationCd']].replace('', pd.NA)
unique_df_sorted


,movieCd,movieNm,multiMovieYn,repNationCd,audiCnt,rank
22,20203702,노량: 죽음의 바다,N,K,290717.0,1.0
2,20212866,서울의 봄,N,K,262422.0,2.0
21,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,N,F,58239.0,3.0
14,20235735,아쿠아맨과 로스트 킹덤,N,F,49368.0,4.0
1,20235596,트롤: 밴드 투게더,N,F,28988.0,5.0
7,20234673,뽀로로 극장판 슈퍼스타 대모험,N,K,11586.0,6.0
20,20235290,도티와 영원의 탑,N,K,10704.0,7.0
15,20234114,괴물,Y,F,8710.0,8.0
0,20236045,"바다 탐험대 옥토넛 어보브 앤 비욘드: 버드, 옥토경보를 울려라!",Y,F,6471.0,9.0
23,20235980,말하고 싶은 비밀,N,F,3765.0,10.0


In [20]:
df=pd.read_parquet("/home/jacob/data/movies/merge/dailyboxoffice/dt=20240104")
df

,movieCd,movieNm,multiMovieYn,repNationCd,audiCnt,rank
18,20235834,위시,N,F,68674,1.0
20,20212866,서울의 봄,N,K,45359,2.0
23,20203702,노량: 죽음의 바다,N,K,44860,3.0
12,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,N,F,12113,4.0
1,20228555,더 퍼스트 슬램덩크,N,F,8114,5.0
7,20235735,아쿠아맨과 로스트 킹덤,N,F,7170,6.0
21,20235583,인투 더 월드,N,F,5898,7.0
13,20234114,괴물,Y,F,3494,8.0
16,20235596,트롤: 밴드 투게더,N,F,3119,9.0
4,20234673,뽀로로 극장판 슈퍼스타 대모험,N,K,3051,10.0


In [21]:
df2=pd.read_parquet("/home/jacob/data/movies/merge/dailyboxoffice/dt=20240101/merged.parquet")
df2

,movieCd,movieNm,multiMovieYn,repNationCd,audiCnt,rank
24,20203702,노량: 죽음의 바다,N,K,290717,1.0
17,20212866,서울의 봄,N,K,262422,2.0
4,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,N,F,58239,3.0
21,20235735,아쿠아맨과 로스트 킹덤,N,F,49368,4.0
16,20235596,트롤: 밴드 투게더,N,F,28988,5.0
1,20234673,뽀로로 극장판 슈퍼스타 대모험,N,K,11586,6.0
3,20235290,도티와 영원의 탑,N,K,10704,7.0
7,20234114,괴물,Y,F,8710,8.0
10,20236045,"바다 탐험대 옥토넛 어보브 앤 비욘드: 버드, 옥토경보를 울려라!",Y,F,6471,9.0
11,20235980,말하고 싶은 비밀,N,F,3765,10.0


In [25]:
df3=pd.read_parquet("/home/jacob/data/movies/dailyboxoffice")
df3.columns

Index(['rnum', 'rank', 'rankInten', 'rankOldAndNew', 'movieCd', 'movieNm',
       'openDt', 'salesAmt', 'salesShare', 'salesInten', 'salesChange',
       'salesAcc', 'audiCnt', 'audiInten', 'audiChange', 'audiAcc', 'scrnCnt',
       'showCnt', 'dt', 'multiMovieYn', 'repNationCd'],
      dtype='object')

In [28]:
df3.to_parquet("~/temp/t_p_1", partition_cols=['openDt','movieCd'])

In [29]:
df3.to_parquet("~/temp/t_p_2", partition_cols=['openDt','salesAmt','repNationCd'])

In [33]:
tp1=pd.read_parquet("/home/jacob/temp/t_p_1/openDt=2024-03-07")
tp1

,rnum,rank,rankInten,rankOldAndNew,movieNm,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,dt,multiMovieYn,repNationCd,movieCd
0,9,9,0,NEW,부활 그 소망,1056000,0.0,1056000,100.0,30697000,96,96,100.0,2932,1,1,20240106,NaN,K,20235114
1,9,9,0,OLD,부활 그 소망,2272000,0.1,1216000,115.2,32969000,215,119,124.0,3147,4,4,20240107,NaN,K,20235114
2,9,9,3,OLD,부활 그 소망,2041000,0.1,-515000,-20.1,39836000,193,-3,-1.5,3781,3,3,20240114,NaN,K,20235114


In [48]:
import pandas as pd

A = pd.DataFrame({
    "movieCd": ["1001", "1002", "1003"],
    "multiMovieYn": ["Y", "Y", "N"],
    "repNationCd": ["K", "F", None]
})

B = pd.DataFrame({
    "movieCd": ["1001", "1003", "1004"],
    "multiMovieYn": [None, "Y", "Y"],
    "repNationCd": [None, "F", "K"]
})
A.set_index("movieCd", inplace=True)
B.set_index("movieCd", inplace=True)
r_df=A.combine_first(B)
r_df
r_df.reset_index(inplace=True)
r_df

,movieCd,multiMovieYn,repNationCd
0,1001,Y,K
1,1002,Y,F
2,1003,N,F
3,1004,Y,K


In [4]:
import pandas as pd
df12=pd.read_parquet("/home/jacob/data/movies/merge/")



In [5]:
print(len(df12))

9020


In [8]:
df12.tail(50)

,movieCd,movieNm,multiMovieYn,repNationCd,audiCnt,rank,dt
17,20228796,하얼빈,N,K,157539,1.0,20241230
23,20200142,소방관,N,K,45563,2.0,20241230
4,20240193,무파사: 라이온 킹,N,F,20264,3.0,20241230
19,20245977,극장판 짱구는 못말려: 우리들의 공룡일기,N,F,16460,4.0,20241230
14,20240737,모아나 2,N,F,15931,5.0,20241230
0,20247693,위키드,N,F,14177,6.0,20241230
3,20244762,서브스턴스,Y,F,4589,7.0,20241230
11,20246201,니코: 오로라 원정대의 모험,N,F,3258,8.0,20241230
16,20247017,대가족,N,K,3064,9.0,20241230
8,20246777,더 폴: 디렉터스 컷,N,F,2884,10.0,20241230


In [22]:
df13=pd.read_parquet("/home/jacob/data/movies/merge/dailyboxoffice")
df13.head(50)

,movieCd,movieNm,multiMovieYn,repNationCd,audiCnt,rank,dt
13,20203702,노량: 죽음의 바다,N,K,290717,1.0,20240101
21,20212866,서울의 봄,N,K,262422,2.0,20240101
12,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,N,F,58239,3.0,20240101
0,20235735,아쿠아맨과 로스트 킹덤,N,F,49368,4.0,20240101
5,20235596,트롤: 밴드 투게더,N,F,28988,5.0,20240101
23,20234673,뽀로로 극장판 슈퍼스타 대모험,N,K,11586,6.0,20240101
22,20235290,도티와 영원의 탑,N,K,10704,7.0,20240101
10,20234114,괴물,Y,F,8710,8.0,20240101
11,20236045,"바다 탐험대 옥토넛 어보브 앤 비욘드: 버드, 옥토경보를 울려라!",Y,F,6471,9.0,20240101
16,20235980,말하고 싶은 비밀,N,F,3765,10.0,20240101


In [16]:
df13=pd.read_parquet("/home/jacob/data/movies/dailyboxoffice/dt=20241231/multiMovieYn=Y")
df14=pd.read_parquet("/home/jacob/data/movies/dailyboxoffice/dt=20241231/multiMovieYn=Y/0bd745355cc14cc9a7d4ce779b8c15fb-0.parquet")
df14


,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,NEW,20244850,시빌 워: 분열의 시대,2024-12-31,217419160,65.4,217419160,100.0,226802160,22039,22039,100.0,22767,662,1546
1,2,2,-1,OLD,20244762,서브스턴스,2024-12-11,40301600,12.1,-2721220,-6.3,1520497600,4228,-361,-7.9,156214,159,192
2,3,3,-1,OLD,20050082,이터널 선샤인,2005-11-10,11408400,3.4,48200,0.4,4206156092,1366,3,0.2,545016,39,46
3,4,4,-1,OLD,20246268,이처럼 사소한 것들,2024-12-11,13182200,4.0,3116800,31.0,369902460,1292,159,14.0,39042,35,47
4,5,5,-1,OLD,20247430,퍼스트레이디,2024-12-12,4931800,1.5,-5293100,-51.8,773313180,584,-538,-48.0,80010,53,67
5,6,6,0,NEW,20020020,밀레니엄 맘보,2003-05-30,5078300,1.5,5078300,100.0,20852300,560,560,100.0,2568,32,34
6,7,7,0,NEW,20241180,"괜찮아, 앨리스",2024-11-13,3863000,1.2,3863000,100.0,191214000,440,440,100.0,21361,2,2
7,8,8,-1,OLD,20247397,존 오브 인터레스트,2024-06-05,3587546,1.1,245246,7.3,2050031040,391,33,9.2,207063,38,38
8,9,9,0,NEW,20040725,화양연화,2000-10-20,3221600,1.0,3221600,100.0,1218845018,313,313,100.0,135365,9,9
9,10,10,-2,OLD,20235926,퍼펙트 데이즈,2024-07-03,2391246,0.7,239946,11.2,1317202231,271,38,16.3,139744,29,29


In [ ]:
import pandas as pd

# Parquet 데이터 경로 (메타 주소 사용)
path1 = "/home/jacob/data/movies/merge/dailyboxoffice/dt=20240102"
path2 = "/home/jacob/data/movie-after/meta/meta.parquet"

# Parquet 데이터 읽기
df1 = pd.read_parquet(path1, engine="pyarrow")
df2 = pd.read_parquet(path2, engine="pyarrow")

# 🔹 1. NaN 값 처리
df1 = df1.fillna("_NAN_")
df2 = df2.fillna("_NAN_")

# 🔹 2. 데이터 타입 통일
df1 = df1.astype(str)
df2 = df2.astype(str)

# 🔹 3. 컬럼 정렬
df1 = df1[sorted(df1.columns)]
df2 = df2[sorted(df2.columns)]

# 🔹 4. 인덱스 정리
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)

# 🔹 5. 차이점 비교
diff_bool = df1 != df2

# 🔹 6. 차이점 출력
if not diff_bool.any().any():
    print("📢 파일이 완전히 동일합니다.")
else:
    print("⚠️ 파일이 다릅니다!")
    diff_data = df1.compare(df2)

In [35]:
kdf=pd.read_parquet("/home/jacob/data/movie-after/meta/meta.parquet")
kdf
column_name = "movieCd"  # 체크할 컬럼명
duplicates = kdf[column_name].duplicated().any()

if duplicates:
    print(f"⚠️ 컬럼 '{column_name}'에 중복된 값이 있습니다.")
else:
    print(f"✅ 컬럼 '{column_name}'에 모든 값이 고유합니다.")

✅ 컬럼 'movieCd'에 모든 값이 고유합니다.


In [15]:
import pandas as pd


path1 = "/home/jacob/data/movies/merge/dailyboxoffice/dt=20240102"
path2 = "/home/jacob/data/movie_after/meta/meta.parquet"
path3 = "/home/jacob/data/movies/merge/dailyboxoffice/dt=20241231"

df1 = pd.read_parquet(path1, engine="pyarrow")
df2 = pd.read_parquet(path2,engine="pyarrow")
df2

,movieCd,movieNm,multiMovieYn,repNationCd,audiCnt,rank
0,19580069,돈,None,K,51,24.0
1,19810057,콰이강의 다리,Y,None,235,21.0
2,19860125,쿼바디스,Y,None,292,19.0
3,20101394,7인의 말괄량이,Y,K,174,21.0
4,20124722,피아니스트의 전설,Y,None,294,18.0
5,20190324,3일의 휴가,N,K,3295,11.0
6,20202247,싱글 인 서울,None,K,212,20.0
7,20203702,노량: 죽음의 바다,N,K,290717,1.0
8,20208446,외계+인 1부,None,K,390,19.0
9,20212350,약속,None,K,130,24.0


In [4]:
import pandas as pd
import pyarrow.dataset as ds

path2 = "/home/jacob/data/movie_after/meta/meta.parquet"
path3 = "/home/jacob/data/movie_after/dailyboxoffice/dt=20240211"
dataset = ds.dataset(path2, format="parquet", partitioning="hive")
df = dataset.to_table().to_pandas()
df


,movieCd,movieNm,multiMovieYn,repNationCd,audiCnt,rank
0,19338006,캐벌케이드,N,None,172,19.0
1,19538021,나이아가라,Y,None,276,20.0
2,19558011,에덴의 동쪽,Y,None,218,21.0
3,19568053,상처뿐인 영광,Y,None,204,20.0
4,19580069,돈,None,K,51,24.0
...,...,...,...,...,...,...
286,20249510,연애 지상주의 구역 Part 2,None,K,92,24.0
287,20249517,애비게일,N,F,5189,3.0
288,20249534,분노의 강,N,K,600,16.0
289,20249673,도뷔시,N,F,201,14.0


In [5]:
path2 = "/home/jacob/data/movie_after/meta/meta.parquet"
path3 = "/home/jacob/data/movie_after/dailyboxoffice/dt=20240301"
dataset = ds.dataset(path3, format="parquet", partitioning="hive")
df = dataset.to_table().to_pandas()
df

,movieCd,movieNm,audiCnt,rank,multiMovieYn,repNationCd
0,20040695,이프 온리,2797,11.0,N,F
1,20124315,엔드 오브 에반게리온,6323,6.0,N,F
2,20161872,너의 이름은.,3298,8.0,N,F
3,20167904,라라랜드,2140,12.0,N,F
4,20228555,더 퍼스트 슬램덩크,8114,5.0,N,F
...,...,...,...,...,...,...
118,20236474,넥스트 골 윈즈,655,20.0,Y,Unclassified
119,20236697,장인과 사위,157,24.0,Y,Unclassified
120,20236890,클럽 제로,1033,19.0,Y,Unclassified
121,20247053,메이 디셈버,508,19.0,Y,Unclassified


In [6]:
pd.read_parquet(path3)

,movieCd,movieNm,audiCnt,rank,multiMovieYn,repNationCd
0,20040695,이프 온리,2797,11.0,N,F
1,20124315,엔드 오브 에반게리온,6323,6.0,N,F
2,20161872,너의 이름은.,3298,8.0,N,F
3,20167904,라라랜드,2140,12.0,N,F
4,20228555,더 퍼스트 슬램덩크,8114,5.0,N,F
...,...,...,...,...,...,...
118,20236474,넥스트 골 윈즈,655,20.0,Y,Unclassified
119,20236697,장인과 사위,157,24.0,Y,Unclassified
120,20236890,클럽 제로,1033,19.0,Y,Unclassified
121,20247053,메이 디셈버,508,19.0,Y,Unclassified


In [40]:
df=pd.read_parquet('~/data/movie_after/meta/meta.parquet')
is_duplicated = df["movieCd"].duplicated().any()
print("중복 있음" if is_duplicated else "중복 없음")
print(len(df))

중복 없음
689


In [10]:
pd.read_parquet('~/data/movies/dailyboxoffice/dt=20240101')

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,multiMovieYn,repNationCd
0,1,1,0,OLD,20203702,노량: 죽음의 바다,2023-12-20,2893509165,38.8,-686119312,-19.2,36926118105,290717,-59184,-16.9,3728850,1706,5793,NaN,NaN
1,2,2,0,OLD,20212866,서울의 봄,2023-11-22,2666429137,35.8,-649456909,-19.6,118016920547,262422,-64054,-19.6,12117201,1614,5444,NaN,NaN
2,3,3,0,OLD,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,2023-12-22,565417320,7.6,-69888800,-11.0,6071745009,58239,-6751,-10.4,626508,834,1671,NaN,NaN
3,4,4,0,OLD,20235735,아쿠아맨과 로스트 킹덤,2023-12-20,507984981,6.8,-156026099,-23.5,7967584753,49368,-14686,-22.9,772354,832,1611,NaN,NaN
4,5,5,0,OLD,20235596,트롤: 밴드 투게더,2023-12-20,273538754,3.7,-16945147,-5.8,3383181955,28988,-1952,-6.3,365927,660,981,NaN,NaN
5,6,6,1,OLD,20234673,뽀로로 극장판 슈퍼스타 대모험,2023-12-13,113680587,1.5,2278496,2.0,3046919883,11586,190,1.7,332776,335,432,NaN,NaN
6,7,7,-1,OLD,20235290,도티와 영원의 탑,2023-12-27,100024717,1.3,-13487835,-11.9,488186693,10704,-1268,-10.6,52400,357,419,NaN,NaN
7,8,8,0,OLD,20234114,괴물,2023-11-29,92381680,1.2,-3768994,-3.9,3884333955,8710,-524,-5.7,396137,187,263,NaN,NaN
8,9,9,0,OLD,20236045,"바다 탐험대 옥토넛 어보브 앤 비욘드: 버드, 옥토경보를 울려라!",2023-12-27,53094530,0.7,-707720,-1.3,372974290,6471,67,1.0,45294,207,248,NaN,NaN
9,10,10,1,OLD,20235980,말하고 싶은 비밀,2023-12-13,38025184,0.5,-2349427,-5.8,868379725,3765,-164,-4.2,89597,95,127,NaN,NaN


In [24]:
pd.read_parquet('/home/jacob/data/movie_after/dailyboxoffice/dt=20240220')

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,multiMovieYn,repNationCd
0,1,1,0,OLD,20236180,웡카,2024-01-31,542454770,34.9,-16847391,-3.0,25073169564,57938,-1325,-2.2,2588220,1116,4095,N,F
1,3,3,0,OLD,20247076,"귀멸의 칼날: 인연의 기적, 그리고 합동 강화 훈련으로",2024-02-14,186107900,12.0,-34541800,-15.7,3588471200,16785,-3219,-16.1,306428,373,1588,N,F
2,10,10,-2,OLD,20234581,아가일,2024-02-07,15270434,1.0,-152172,-1.0,1426625820,1636,11,0.7,138800,318,400,N,F
3,1,1,0,OLD,20236180,웡카,2024-01-31,542454770,51.4,-16847391,-3.0,25073169564,57938,-1325,-2.2,2588220,1116,4095,N,F
4,2,2,0,OLD,20247076,"귀멸의 칼날: 인연의 기적, 그리고 합동 강화 훈련으로",2024-02-14,186107900,17.6,-34541800,-15.7,3588471200,16785,-3219,-16.1,306428,373,1588,N,F
5,7,7,-1,OLD,20234581,아가일,2024-02-07,15270434,1.4,-152172,-1.0,1426625820,1636,11,0.7,138800,318,400,N,F
6,8,8,0,OLD,20246989,스노우 퍼핀즈,2024-02-15,10869334,1.0,3349117,44.5,114256763,1377,448,48.2,13583,187,196,N,F
7,9,9,-2,OLD,20247227,우견니,2024-02-14,7314051,0.7,-982073,-11.8,115736492,944,-239,-20.2,14521,121,239,N,F
8,1,1,0,OLD,20236180,웡카,2024-01-31,542454770,63.6,-16847391,-3.0,25073169564,57938,-1325,-2.2,2588220,1116,4095,N,F
9,2,2,0,OLD,20247076,"귀멸의 칼날: 인연의 기적, 그리고 합동 강화 훈련으로",2024-02-14,186107900,21.8,-34541800,-15.7,3588471200,16785,-3219,-16.1,306428,373,1588,N,F


In [27]:
pd.read_parquet('/home/jacob/data/movies/dailyboxoffice/dt=20240220')

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,multiMovieYn,repNationCd
0,1,1,0,OLD,20236180,웡카,2024-01-31,542454770,34.9,-16847391,-3.0,25073169564,57938,-1325,-2.2,2588220,1116,4095,NaN,NaN
1,2,2,0,OLD,20247074,건국전쟁,2024-02-01,340178230,21.9,-35533394,-9.5,7474549692,38919,-3654,-8.6,792001,843,2772,NaN,NaN
2,3,3,0,OLD,20247076,"귀멸의 칼날: 인연의 기적, 그리고 합동 강화 훈련으로",2024-02-14,186107900,12.0,-34541800,-15.7,3588471200,16785,-3219,-16.1,306428,373,1588,NaN,NaN
3,4,4,0,OLD,20206946,시민덕희,2024-01-24,116309105,7.5,-9350854,-7.4,15750290914,12412,-812,-6.1,1665759,697,1871,NaN,NaN
4,5,5,0,OLD,20231415,소풍,2024-02-07,80983208,5.2,7880289,10.8,2405415430,9680,1082,12.6,266971,601,1061,NaN,NaN
5,6,6,0,OLD,20212668,도그데이즈,2024-02-07,62810285,4.0,1691816,2.8,3436328499,6461,36,0.6,353291,591,1044,NaN,NaN
6,7,7,0,NEW,20234675,파묘,2024-02-22,49104000,3.2,49104000,100.0,49104000,5173,5173,100.0,5173,20,20,NaN,NaN
7,8,8,-1,OLD,20211477,데드맨,2024-02-07,19875906,1.3,-4618901,-18.9,2255855548,2110,-608,-22.4,236304,387,524,NaN,NaN
8,9,9,2,OLD,20246966,아기상어 극장판: 사이렌 스톤의 비밀,2024-02-07,15736278,1.0,8274739,110.9,737169431,2023,1091,117.1,83188,173,182,NaN,NaN
9,10,10,-2,OLD,20234581,아가일,2024-02-07,15270434,1.0,-152172,-1.0,1426625820,1636,11,0.7,138800,318,400,NaN,NaN


In [25]:
pd.read_parquet('/home/jacob/data/movie_after/dailyboxoffice/dt=20240221')

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,multiMovieYn,repNationCd
0,1,1,0,OLD,20236180,웡카,2024-01-31,544528944,32.0,2074174,0.4,25617698508,58643,705,1.2,2646863,1156,4269,N,F
1,3,3,0,OLD,20247076,"귀멸의 칼날: 인연의 기적, 그리고 합동 강화 훈련으로",2024-02-14,204959600,12.1,18851700,10.1,3793430800,19302,2517,15.0,325730,377,1550,N,F
2,6,6,0,NEW,20247219,사운드 오브 프리덤,2024-02-21,86918300,5.1,86918300,100.0,108250500,8148,8148,100.0,10206,248,710,N,F
3,1,1,0,OLD,20236180,웡카,2024-01-31,544528944,46.5,2074174,0.4,25617698508,58643,705,1.2,2646863,1156,4269,N,F
4,2,2,0,OLD,20247076,"귀멸의 칼날: 인연의 기적, 그리고 합동 강화 훈련으로",2024-02-14,204959600,17.5,18851700,10.1,3793430800,19302,2517,15.0,325730,377,1550,N,F
5,4,4,0,NEW,20247219,사운드 오브 프리덤,2024-02-21,86918300,7.4,86918300,100.0,108250500,8148,8148,100.0,10206,248,710,N,F
6,8,8,0,OLD,20246989,스노우 퍼핀즈,2024-02-15,14506000,1.2,3636666,33.5,128762763,1792,415,30.1,15375,135,151,N,F
7,1,1,0,OLD,20236180,웡카,2024-01-31,544528944,49.4,2074174,0.4,25617698508,58643,705,1.2,2646863,1156,4269,N,F
8,2,2,0,OLD,20247076,"귀멸의 칼날: 인연의 기적, 그리고 합동 강화 훈련으로",2024-02-14,204959600,18.6,18851700,10.1,3793430800,19302,2517,15.0,325730,377,1550,N,F
9,3,3,0,NEW,20247219,사운드 오브 프리덤,2024-02-21,86918300,7.9,86918300,100.0,108250500,8148,8148,100.0,10206,248,710,N,F


In [26]:
pd.read_parquet('/home/jacob/data/movies/dailyboxoffice/dt=20240221')

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,multiMovieYn,repNationCd
0,1,1,0,OLD,20236180,웡카,2024-01-31,544528944,32.0,2074174,0.4,25617698508,58643,705,1.2,2646863,1156,4269,NaN,NaN
1,2,2,0,OLD,20247074,건국전쟁,2024-02-01,303783659,17.9,-36394571,-10.7,7778333351,34787,-4132,-10.6,826788,821,2860,NaN,NaN
2,3,3,0,OLD,20247076,"귀멸의 칼날: 인연의 기적, 그리고 합동 강화 훈련으로",2024-02-14,204959600,12.1,18851700,10.1,3793430800,19302,2517,15.0,325730,377,1550,NaN,NaN
3,4,4,0,OLD,20206946,시민덕희,2024-01-24,108252189,6.4,-8056916,-6.9,15858543103,11598,-814,-6.6,1677357,645,1656,NaN,NaN
4,5,5,0,OLD,20231415,소풍,2024-02-07,75273956,4.4,-5709252,-7.0,2480689386,8960,-720,-7.4,275931,557,967,NaN,NaN
5,6,6,0,NEW,20247219,사운드 오브 프리덤,2024-02-21,86918300,5.1,86918300,100.0,108250500,8148,8148,100.0,10206,248,710,NaN,NaN
6,7,7,-1,OLD,20212668,도그데이즈,2024-02-07,48381468,2.8,-14428817,-23.0,3484709967,5228,-1233,-19.1,358519,460,760,NaN,NaN
7,8,8,0,NEW,20234641,극장판 츠루네: 시작의 한 발,2024-02-21,32836527,1.9,32836527,100.0,42682527,3654,3654,100.0,4654,119,275,NaN,NaN
8,9,9,60,OLD,20247063,바튼 아카데미,2024-02-21,30020102,1.8,29633402,7663.2,64293802,3318,3284,9658.8,6430,254,412,NaN,NaN
9,10,10,0,NEW,20247444,월레스와 그로밋 더 클래식 컬렉션,2024-02-21,47812500,2.8,47812500,100.0,47812500,3190,3190,100.0,3190,47,75,NaN,NaN


In [19]:
pd.read_parquet('/home/jacob/data/movie_after/dailyboxoffice/')

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,...,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,dt,multiMovieYn,repNationCd
0,3,3,0,OLD,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,2023-12-22,565417320,7.6,-69888800,...,6071745009,58239,-6751,-10.4,626508,834,1671,20240101,N,F
1,4,4,0,OLD,20235735,아쿠아맨과 로스트 킹덤,2023-12-20,507984981,6.8,-156026099,...,7967584753,49368,-14686,-22.9,772354,832,1611,20240101,N,F
2,5,5,0,OLD,20235596,트롤: 밴드 투게더,2023-12-20,273538754,3.7,-16945147,...,3383181955,28988,-1952,-6.3,365927,660,981,20240101,N,F
3,10,10,1,OLD,20235980,말하고 싶은 비밀,2023-12-13,38025184,0.5,-2349427,...,868379725,3765,-164,-4.2,89597,95,127,20240101,N,F
4,3,3,0,OLD,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,2023-12-22,565417320,7.8,-69888800,...,6071745009,58239,-6751,-10.4,626508,834,1671,20240101,N,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,8,8,6,OLD,20246953,검은 소년,2024-02-07,5025400,0.8,2080700,...,32057939,617,273,79.4,3811,29,42,20240213,Y,K
2206,1,1,1,OLD,20247074,건국전쟁,2024-02-01,467599520,45.5,-392646620,...,3726694567,52207,-36257,-41.0,382141,580,1308,20240213,Y,K
2207,4,4,0,OLD,20231415,소풍,2024-02-07,109247918,10.6,-233135928,...,1713234643,12915,-25265,-66.2,187360,651,1413,20240213,Y,K
2208,7,7,2,OLD,20226638,아네모네,2024-02-07,5538117,0.5,-599283,...,54825951,671,-10,-1.5,6458,43,49,20240213,Y,K


In [41]:
pd.read_parquet('/home/jacob/data/movie_after/dailyboxoffice/dt=20241231')

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,multiMovieYn,repNationCd
0,5,5,-2,OLD,20240193,무파사: 라이온 킹,2024-12-18,196176840,4.4,5704420,3.0,6543943550,21666,1402,6.9,656645,628,951,N,F
1,6,6,-1,OLD,20240737,모아나 2,2024-11-27,169266590,3.8,24472230,16.9,31118051070,18863,2932,18.4,3372815,537,839,N,F
2,7,7,-3,OLD,20245977,극장판 짱구는 못말려: 우리들의 공룡일기,2024-12-18,157173860,3.5,5440680,3.6,5346417380,17213,753,4.6,567725,545,950,N,F
3,8,8,-2,OLD,20247693,위키드,2024-11-20,145609200,3.3,13840860,10.5,20277702850,15440,1263,8.9,2059163,371,553,N,F
4,10,10,0,OLD,20246777,더 폴: 디렉터스 컷,2024-12-25,27669400,0.6,-243900,-0.9,299843400,2820,-64,-2.2,27865,77,114,N,F
5,4,4,-1,OLD,20240193,무파사: 라이온 킹,2024-12-18,196176840,4.8,5704420,3.0,6543943550,21666,1402,6.9,656645,628,951,N,F
6,5,5,0,OLD,20240737,모아나 2,2024-11-27,169266590,4.1,24472230,16.9,31118051070,18863,2932,18.4,3372815,537,839,N,F
7,6,6,-2,OLD,20245977,극장판 짱구는 못말려: 우리들의 공룡일기,2024-12-18,157173860,3.8,5440680,3.6,5346417380,17213,753,4.6,567725,545,950,N,F
8,7,7,-1,OLD,20247693,위키드,2024-11-20,145609200,3.5,13840860,10.5,20277702850,15440,1263,8.9,2059163,371,553,N,F
9,8,8,1,OLD,20246777,더 폴: 디렉터스 컷,2024-12-25,27669400,0.7,-243900,-0.9,299843400,2820,-64,-2.2,27865,77,114,N,F
